In [1]:
import os
import sklearn
import numpy as np
import pandas as pd
import tensorflow
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint ,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, LSTM, Bidirectional, Input, Lambda, GRU
from tensorflow.keras.layers import Permute,GlobalMaxPool1D,Concatenate, BatchNormalization, Dropout, GlobalAveragePooling1D, GlobalMaxPool2D
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report, accuracy_score, log_loss
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline

epoch_num = 2

#Enter '0' if you want to train a new model,
#'1' to use a pretrained model and 
#'2' to train a pretraind model
pDenseNet = 0
pDenseNet_XceptionNet = 0
pDenseNet_XceptionNet_ResNet = 0
pDenseNet_XceptionNet_ResNet_BiGRU = 0
pDenseNet_XceptionNet_ResNet_BiLSTM = 0
pDenseNet_BiGRU = 0
pDenseNet_BiLSTM = 0
pDenseNet_Xception_BiGRU = 0
pDenseNet_Xception_BiLSTM = 0

In [2]:
path = '/kaggle/input/mushrooms-classification-common-genuss-images/mushrooms/Mushrooms/'
df = pd.DataFrame()
for imlabel in os.listdir(path):
    for image in os.listdir(path+imlabel):
        df = df.append({"path" : path+imlabel+"/"+image, "label" : imlabel}, ignore_index=True)

df.head()

## Splitting Data :
70%, 20%, 10% for training, validation and testing respecitvely

In [3]:
train_X, val_test_X, train_Y, val_test_Y = train_test_split(df['path'], df['label'], test_size=0.3, shuffle=True)
train_data = pd.concat({'path':train_X, 'label':train_Y}, axis=1)

val_X, test_X, val_Y, test_Y = train_test_split(val_test_X, val_test_Y, test_size=0.33, shuffle=True)

validation_data = pd.concat({'path':val_X, 'label':val_Y}, axis=1)
test_data = pd.concat({'path':test_X, 'label':test_Y}, axis=1)

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
vertical_flip = True)
train_data_generator = train_datagen.flow_from_dataframe(train_data,
                                                         x_col = 'path',
                                                         y_col = 'label',
                                                        batch_size=16,
                                                         color_mode="rgb",
                                                         target_size=(224, 224),
                                                        class_mode="categorical")
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_data_generator = validation_datagen.flow_from_dataframe(validation_data,
                                                                   x_col = 'path',
                                                                   y_col = 'label',
                                                                   batch_size=16,
                                                                   color_mode="rgb",
                                                                   target_size=(224, 224),
                                                                   class_mode="categorical")

test_datagen = ImageDataGenerator(rescale=1./255)
test_data_generator = test_datagen.flow_from_dataframe(test_data,
                                         x_col = 'path',
                                          y_col = 'label',
                                          batch_size=test_data['path'].shape[0],
                                          color_mode="rgb",
                                          target_size=(224, 224),
                                          class_mode="categorical")

# Funtion definitions for important stuff.

In [5]:
def model_fitting(model, model_name, p):
    model_path = model_name+'/model'
    history_path = model_name+'/model_history.npy'

    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=0.09,patience=2)

    if ((not os.path.exists(model_path)) or (not os.path.exists(history_path))):
        history = model.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[reduce_lr])
        history = history.history
        model.save(model_path)
        np.save(history_path,history)

    else:
        if (p==0):
            history = model.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[reduce_lr])
            history = history.history
            np.save(history_path,history)
            model.save(model_path)
        elif (p==1):
            model = tensorflow.keras.models.load_model(model_path)
            history=np.load(history_path,allow_pickle='TRUE').item()
        elif (p==2):
            model = tensorflow.keras.models.load_model(model_path)
            history = model.fit(train_data_generator, epochs=epoch_num, validation_data=validation_data_generator, callbacks=[reduce_lr])
            history = history.history
            np.save(history_path,history)
            model.save(model_path)
    return history

In [6]:
def plot_model_cache(history, model_name):
    accuracy = history['accuracy']
    val_accuracy = history['val_accuracy']
    loss = history['loss']
    val_loss = history['val_loss']
    precision = history['precision']
    val_precision = history['val_precision']
    recall = history['recall']
    val_recall = history['val_recall']
    f1_score = 2 * (np.array(precision) * np.array(recall)) / (np.array(precision) + np.array(recall))
    val_f1_score = 2 * (np.array(val_precision) * np.array(val_recall)) / (np.array(val_precision) + np.array(val_recall))

    plt.plot(range(epoch_num), accuracy, 'r', label='Training accuracy')
    plt.plot(range(epoch_num), val_accuracy, 'b', label='Validation accuracy')
    plt.title(model_name + ' Training and validation accuracy')
    plt.legend()
    plt.savefig(model_name+'_train_val_acc_plot.jpg')

    plt.figure()

    plt.plot(range(epoch_num), loss, 'r', label='Training loss')
    plt.plot(range(epoch_num), val_loss, 'b', label='Validation loss')
    plt.ylim(top=7, bottom=0)
    plt.title(model_name + ' Training and validation loss')
    plt.legend()
    plt.savefig(model_name+'_trian_val_loss_plot.jpg')

    plt.figure()

    plt.plot(range(epoch_num), precision, 'r', label='Training Precision')
    plt.plot(range(epoch_num), val_precision, 'b', label='Validation Precision')
    plt.title(model_name + ' Training and validation Precision')
    plt.legend()
    plt.savefig(model_name+'_trian_val_precision_plot.jpg')

    plt.figure()

    plt.plot(range(epoch_num), recall, 'r', label='Training Recall')
    plt.plot(range(epoch_num), val_recall, 'b', label='Validation Recall')
    plt.title(model_name + ' Training and validation Recall')
    plt.legend()
    plt.savefig(model_name+'_trian_val_recall_plot.jpg')

    plt.figure()
    plt.plot(range(epoch_num), f1_score, 'r', label='Training F1_Score')
    plt.plot(range(epoch_num), val_f1_score, 'b', label='Validation F1_Score')
    plt.title(model_name + ' Training and validation F1_score')
    plt.legend()
    plt.savefig(model_name+'_trian_val_f1_score_plot.jpg')

In [7]:
def print_test_cache(model):

    for image, lab in test_data_generator:
        predict_y = model.predict(image)
        real_lab = lab
    
        break;

    print(f"Test Accuracy  :  {accuracy_score(np.argmax(real_lab, axis=1), np.argmax(predict_y, axis=1))}")
    print(f"Test Loss      :  {sklearn.metrics.zero_one_loss(np.argmax(real_lab, axis=1), np.argmax(predict_y, axis=1))}")
    print(f"Test Precision :  {precision_score(np.argmax(real_lab, axis=1), np.argmax(predict_y, axis=1), average='macro')}")
    print(f"Test Recall    :  {recall_score(np.argmax(real_lab, axis=1), np.argmax(predict_y, axis=1), average='macro')}")
    print(f"Test F1_Score  :  {f1_score(np.argmax(real_lab, axis=1), np.argmax(predict_y, axis=1), average='macro')}")
    
    cnf = tensorflow.math.confusion_matrix(np.argmax(real_lab, axis=1), np.argmax(predict_y, axis=1))
    c_plot = sns.heatmap(cnf, annot=True)
    c_plot.set(xlabel='true_value', ylabel='prediction')

In [8]:
def show_predicted_image(i, model):
    for real_image, lab in test_data_generator:
        image = real_image
        real_lab = lab
        break
    
    predict_y = model.predict(image)
    plt.imshow(image[i])
    plt.title(os.listdir(path)[np.argmax(predict_y[i])])
    plt.show()
    if (np.argmax(predict_y[i])==np.argmax(real_lab[i])):
        print("True prediction")
    else:
        print("Wrong value prediction")
        print(f"Correct label is : {os.listdir(path)[np.argmax(real_lab[i])]}")

In [9]:
def print_classification_report(model):
    for real_image, lab in test_data_generator:
        image = real_image
        real_lab = lab
        break
    
    predict_y = model.predict(image)
    print(classification_report(np.argmax(real_lab, axis=1), np.argmax(predict_y, axis=1)))

In [10]:
def cmp_plot(history_list, model_list, entity, name, epoch_num, train=True, validation=True):
    plt.figure()
    i=0
    
    for history in history_list:
        if train:
            plt.plot(range(epoch_num), history[entity], label=f'model{i+1} Training {entity}')
        if validation:
            plt.plot(range(epoch_num), history['val_'+entity], label=f'model{i+1} validation {entity}')
        i+=1
    plt.title(name+' Plot')
    plt.legend()
    plt.savefig(name+'plot.png')
    plt.show()

#  DenseNet implementation

In [27]:
densenet1 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet1(img_inputs)
x = tensorflow.keras.layers.GlobalAveragePooling2D()(x)
x = tensorflow.keras.layers.Dense(128, activation='relu')(x)

m_output = tensorflow.keras.layers.Dense(9, activation='softmax')(x)

model1 = tensorflow.keras.Model(inputs=img_inputs, outputs=m_output)

model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model1.summary()

In [28]:
tensorflow.keras.utils.plot_model(model1, to_file='/denseNet.png')

In [29]:
history1 = model_fitting(model1, 'DenseNet', pDenseNet)

In [30]:
plot_model_cache(history1, 'DenseNet')

### DenseNet Model performance on Test data.

In [31]:
print_test_cache(model1)

In [32]:
show_predicted_image(7, model1)

In [33]:
show_predicted_image(237, model1)

## Classification report for test datset

In [34]:
print_classification_report(model1)

#  DenseNet + XceptionNet implementation

In [35]:
densenet2 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

xceptionNet2 = tensorflow.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet2(img_inputs)
x = tensorflow.keras.layers.GlobalAveragePooling2D()(x)
x = tensorflow.keras.layers.Dense(128, activation='relu')(x)

y = xceptionNet2(img_inputs)
y = tensorflow.keras.layers.GlobalAveragePooling2D()(y)
y = tensorflow.keras.layers.Dense(128, activation='relu')(y)

y = tensorflow.keras.layers.Concatenate()([x, y])
y = tensorflow.keras.layers.Dense(128, activation="relu")(y)

m_output = tensorflow.keras.layers.Dense(9, activation='softmax')(y)

model2 = tensorflow.keras.Model(inputs=img_inputs, outputs=m_output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model2.summary()

In [36]:
tensorflow.keras.utils.plot_model(model2, to_file="DenseNet_XceptionNet.png")

In [37]:
history2 = model_fitting(model2, 'DenseNet_XceptionNet', pDenseNet_XceptionNet)

In [38]:
plot_model_cache(history2, 'DenseNet_XceptionNet')

## DenseNet+XceptionNet Performance on Test data.

In [39]:
print_test_cache(model2)

In [40]:
show_predicted_image(7, model2)

In [41]:
show_predicted_image(237, model2)

In [42]:
print_classification_report(model2)

# DenseNet + XceptionNet + ResNet implementation

In [43]:
densenet3 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

xceptionNet3 = tensorflow.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

resNet3 = tensorflow.keras.applications.resnet.ResNet101(
    include_top=False, weights='imagenet',
    input_shape=(224, 224, 3)
)


img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet3(img_inputs)
x = tensorflow.keras.layers.GlobalAveragePooling2D()(x)
x = tensorflow.keras.layers.Dense(128, activation='relu')(x)

y = xceptionNet3(img_inputs)
y = tensorflow.keras.layers.GlobalAveragePooling2D()(y)
y = tensorflow.keras.layers.Dense(128, activation='relu')(y)

z = resNet3(img_inputs)
z = tensorflow.keras.layers.GlobalAveragePooling2D()(z)
z = tensorflow.keras.layers.Dense(128, activation='relu')(z)

z = tensorflow.keras.layers.Concatenate()([x, y, z])
z = tensorflow.keras.layers.Dense(128, activation="relu")(z)

m_output = tensorflow.keras.layers.Dense(9, activation='softmax')(z)

model3 = tensorflow.keras.Model(inputs=img_inputs, outputs=m_output)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model3.summary()

In [44]:
tensorflow.keras.utils.plot_model(model3, to_file='DenseNet_XceptionNet_ResNet.png')

In [45]:
history3 = model_fitting(model3, 'DenseNet_XceptionNet_ResNet', pDenseNet_XceptionNet_ResNet)

In [46]:
plot_model_cache(history3, 'DenseNet_XceptionNet_ResNet')

In [47]:
print_test_cache(model3)

In [48]:
show_predicted_image(27, model3)

In [49]:
show_predicted_image(77, model3)

In [50]:
print_classification_report(model3)

# Plots for Comparison of Models

## 1. Train Accuracy plot.

In [51]:
history_list1 = [history1, history2, history3]
model_list1 = [model1, model2, model3]

cmp_plot(history_list1, model_list1, 'accuracy', 'Trianing_validation_acc_cmp', epoch_num, train=True, validation=True)

## 2. Training - Validation Loss plot.

In [52]:
cmp_plot(history_list1, model_list1, 'loss', 'Trianing_validation_loss_cmp', epoch_num)

## 3. Training - Validation Precision Plot

In [53]:
cmp_plot(history_list1, model_list1, 'precision', 'Trianing_validation_precision_cmp', epoch_num)

## 4. Training - Validation Recall Plot.

In [54]:
cmp_plot(history_list1, model_list1, 'recall', 'Trianing_validation_recall_cmp', epoch_num)

## All Models comparison for test dataset

In [55]:
for image, lab in test_data_generator:
    cmp = pd.DataFrame({'D' : model1.evaluate(image, lab), 'DX' : model2.evaluate(image, lab), 'DXR' : model3.evaluate(image, lab)}, index=['Loss', 'Accuracy', 'Precision', 'Recall'])
    break

In [56]:
cmp

# Model-4 : DenseNet+XceptionNet+ResNet+BiGRU

In [19]:
densenet4 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

xceptionNet4 = tensorflow.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

resNet4 = tensorflow.keras.applications.resnet.ResNet101(
    include_top=False, weights='imagenet',
    input_shape=(224, 224, 3)
)


img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet4(img_inputs)

y = xceptionNet4(img_inputs)

z = resNet4(img_inputs)

z = tensorflow.keras.layers.Concatenate()([x, y, z])

z = tensorflow.keras.layers.Conv2D(1, (1, 1),activation='relu')(z)

###################  BIDIRECTIONAL-GRU IMPLEMENTATION ###############################

lambda_ = Lambda(lambda x: tensorflow.squeeze(x,axis=3))(z)

lstm_ = Bidirectional(GRU(8,activation='relu',return_sequences=True))(lambda_)

lstm_ = BatchNormalization()(lstm_)
maxpool1 = GlobalMaxPool1D()(lstm_)


dense_1 = Dense(128,activation='relu')(maxpool1)
output_ = Dense(9,activation='softmax')(dense_1)

model4 = tensorflow.keras.Model(inputs=img_inputs, outputs=output_)

model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model4.summary()

In [20]:
tensorflow.keras.utils.plot_model(model4, to_file='DenseNet_XceptionNet_ResNet_GRU.png')

In [21]:
history4 = model_fitting(model4, 'denseNet_XceptionNet_ResNet_BiGRU', pDenseNet_XceptionNet_ResNet_BiGRU)

In [22]:
plot_model_cache(history4, 'denseNet_XceptionNet_ResNet_BiGRU')

In [23]:
print_test_cache(model4)

In [24]:
show_predicted_image(27, model4)

In [25]:
show_predicted_image(347, model4)

In [26]:
print_classification_report(model4)

# Model-5 : DenseNet+XceptionNet+ResNet+BiLSTM

In [11]:
densenet5 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

xceptionNet5 = tensorflow.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

resNet5 = tensorflow.keras.applications.resnet.ResNet101(
    include_top=False, weights='imagenet',
    input_shape=(224, 224, 3)
)


img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet5(img_inputs)

y = xceptionNet5(img_inputs)

z = resNet5(img_inputs)

z = tensorflow.keras.layers.Concatenate()([x, y, z])

z = tensorflow.keras.layers.Conv2D(1, (1, 1),activation='relu')(z)

###################  BIDIRECTIONAL-GRU IMPLEMENTATION ###############################

lambda_ = Lambda(lambda x: tensorflow.squeeze(x,axis=3))(z)

lstm_ = Bidirectional(LSTM(8,activation='relu',return_sequences=True))(lambda_)

lstm_ = BatchNormalization()(lstm_)
maxpool1 = GlobalMaxPool1D()(lstm_)

dense_1 = Dense(128,activation='relu')(maxpool1)
output_ = Dense(9,activation='softmax')(dense_1)

model5 = tensorflow.keras.Model(inputs=img_inputs, outputs=output_)

model5.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model5.summary()

In [12]:
tensorflow.keras.utils.plot_model(model5, to_file='DenseNet_XceptionNet_ResNet_BiLSTM.png')

In [13]:
# tensorflow.keras.backend.clear_session()
history5 = model_fitting(model5, 'DenseNet_XceptionNet_ResNet_BiLSTM', pDenseNet_XceptionNet_ResNet_BiLSTM)

In [14]:
plot_model_cache(history5, 'denseNet_XceptionNet_ResNet_BiGRU')

In [15]:
print_test_cache(model5)

In [16]:
show_predicted_image(27, model5)

In [17]:
show_predicted_image(632, model5)

In [18]:
print_classification_report(model5)

# Model-6 : DenseNet+BiGRU

In [57]:
densenet6 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet6(img_inputs)

z = tensorflow.keras.layers.Conv2D(1, (1, 1),activation='relu')(x)

###################  BIDIRECTIONAL-GRU IMPLEMENTATION ###############################

lambda_ = Lambda(lambda x: tensorflow.squeeze(x,axis=3))(z)

lstm_ = Bidirectional(GRU(8,activation='relu',return_sequences=True))(lambda_)

permute_=Permute((2,1),input_shape=(7, 7))(lambda_)
lstm_2 = Bidirectional(GRU(8,activation='relu',return_sequences=True))(permute_)


lstm_ = BatchNormalization()(lstm_)
maxpool1 = GlobalMaxPool1D()(lstm_)


lstm_2 = BatchNormalization()(lstm_2)
maxpool2 = GlobalMaxPool1D()(lstm_2)


concat_ = Concatenate(axis=1)([maxpool1,maxpool2])
dense_1 = Dense(128,activation='relu')(maxpool1)
output_ = Dense(9,activation='softmax')(dense_1)

model6 = tensorflow.keras.Model(inputs=img_inputs, outputs=output_)

model6.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model6.summary()

In [58]:
tensorflow.keras.utils.plot_model(model6, to_file='DenseNet_BiGRU.png')

In [60]:
history6 = model_fitting(model6, 'DenseNet_BiGRU', pDenseNet_BiGRU)

In [62]:
plot_model_cache(history6, 'DenseNet_BiGRU')

In [63]:
print_test_cache(model6)

In [64]:
show_predicted_image(27, model6)

In [65]:
show_predicted_image(453, model6)

In [66]:
print_classification_report(model6)

# Model-7 : DenseNet+BiLSTM

In [67]:
densenet7 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet7(img_inputs)

z = tensorflow.keras.layers.Conv2D(1, (1, 1),activation='relu')(x)

###################  BIDIRECTIONAL-GRU IMPLEMENTATION ###############################

lambda_ = Lambda(lambda x: tensorflow.squeeze(x,axis=3))(z)

lstm_ = Bidirectional(LSTM(8,activation='relu',return_sequences=True))(lambda_)

permute_=Permute((2,1),input_shape=(7, 7))(lambda_)
lstm_2 = Bidirectional(LSTM(8,activation='relu',return_sequences=True))(permute_)


lstm_ = BatchNormalization()(lstm_)
maxpool1 = GlobalMaxPool1D()(lstm_)


lstm_2 = BatchNormalization()(lstm_2)
maxpool2 = GlobalMaxPool1D()(lstm_2)


concat_ = Concatenate(axis=1)([maxpool1,maxpool2])
dense_1 = Dense(128,activation='relu')(maxpool1)
output_ = Dense(9,activation='softmax')(dense_1)

model7 = tensorflow.keras.Model(inputs=img_inputs, outputs=output_)

model7.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model7.summary()

In [68]:
tensorflow.keras.utils.plot_model(model7, to_file='DenseNet_BiLSTM.png')

In [69]:
history7 = model_fitting(model7, 'DenseNet_BiLSTM', pDenseNet_BiLSTM)

In [70]:
plot_model_cache(history7, 'DenseNet_BiLSTM')

In [71]:
print_test_cache(model7)

In [72]:
show_predicted_image(27, model7)

In [73]:
show_predicted_image(543, model7)

In [74]:
print_classification_report(model7)

# Model-8: DenseNet+XceptionNet+BiGRU

In [75]:
densenet8 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

xceptionNet8 = tensorflow.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)


img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet8(img_inputs)

y = xceptionNet8(img_inputs)

z = tensorflow.keras.layers.Concatenate()([x, y])

z = tensorflow.keras.layers.Conv2D(1, (1, 1),activation='relu')(z)

###################  BIDIRECTIONAL-GRU IMPLEMENTATION ###############################

lambda_ = Lambda(lambda x: tensorflow.squeeze(x,axis=3))(z)

lstm_ = Bidirectional(GRU(8,activation='relu',return_sequences=True))(lambda_)

permute_=Permute((2,1),input_shape=(7, 7))(lambda_)
lstm_2 = Bidirectional(GRU(8,activation='relu',return_sequences=True))(permute_)


lstm_ = BatchNormalization()(lstm_)
maxpool1 = GlobalMaxPool1D()(lstm_)


lstm_2 = BatchNormalization()(lstm_2)
maxpool2 = GlobalMaxPool1D()(lstm_2)


concat_ = Concatenate(axis=1)([maxpool1,maxpool2])
dense_1 = Dense(128,activation='relu')(maxpool1)
output_ = Dense(9,activation='softmax')(dense_1)

model8 = tensorflow.keras.Model(inputs=img_inputs, outputs=output_)

model8.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model8.summary()

In [76]:
tensorflow.keras.utils.plot_model(model8, to_file='DenseNet_Xception_BiGRU.png')

In [77]:
history8 = model_fitting(model8, 'DenseNet_Xception_BiGRU', pDenseNet_Xception_BiGRU)

In [78]:
plot_model_cache(history8, 'DenseNet_Xception_BiGRU')

In [79]:
print_test_cache(model8)

In [80]:
show_predicted_image(27, model8)

In [81]:
show_predicted_image(2, model8)

In [82]:
print_classification_report(model8)

# Model-9: DenseNet+XceptionNet+BiLSTM,

In [83]:
densenet9 = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

xceptionNet9 = tensorflow.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)


img_inputs = tensorflow.keras.Input(shape=(224, 224, 3))

x = densenet9(img_inputs)

y = xceptionNet9(img_inputs)

z = tensorflow.keras.layers.Concatenate()([x, y])

z = tensorflow.keras.layers.Conv2D(1, (1, 1),activation='relu')(z)

###################  BIDIRECTIONAL-GRU IMPLEMENTATION ###############################

lambda_ = Lambda(lambda x: tensorflow.squeeze(x,axis=3))(z)

lstm_ = Bidirectional(LSTM(8,activation='relu',return_sequences=True))(lambda_)

permute_=Permute((2,1),input_shape=(7, 7))(lambda_)
lstm_2 = Bidirectional(LSTM(8,activation='relu',return_sequences=True))(permute_)


lstm_ = BatchNormalization()(lstm_)
maxpool1 = GlobalMaxPool1D()(lstm_)


lstm_2 = BatchNormalization()(lstm_2)
maxpool2 = GlobalMaxPool1D()(lstm_2)


concat_ = Concatenate(axis=1)([maxpool1,maxpool2])
dense_1 = Dense(128,activation='relu')(maxpool1)
output_ = Dense(9,activation='softmax')(dense_1)

model9 = tensorflow.keras.Model(inputs=img_inputs, outputs=output_)

model9.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
model9.summary()

In [84]:
tensorflow.keras.utils.plot_model(model9, to_file='DenseNet_Xception_BiLSTM.png')

In [85]:
history9 = model_fitting(model9, 'DenseNet_Xception_BiLSTM', pDenseNet_Xception_BiLSTM)

In [86]:
plot_model_cache(history9, 'DenseNet_Xception_BiLSTM')

In [87]:
print_test_cache(model9)

In [88]:
show_predicted_image(27, model9)

In [89]:
show_predicted_image(345, model9)

In [90]:
print_classification_report(model9)

# COMPARISON PLOTS:

In [91]:
history_list1 = [history1, history2, history3, history4, history5, history6, history7, history8, history9]
model_list1 = [model1, model2, model3, model4, model5, model6, model7, model8, model9]

cmp_plot(history_list1, model_list1, 'accuracy', 'Trianing_acc_cmp', epoch_num, validation=False)

In [92]:

cmp_plot(history_list1, model_list1, 'accuracy', 'validation_acc_cmp', epoch_num, train=False)

In [93]:
cmp_plot(history_list1, model_list1, 'loss', 'Trianing_loss_cmp', epoch_num, validation=False)

In [94]:
cmp_plot(history_list1, model_list1, 'loss', 'Validation_loss_cmp', epoch_num, train=False)

In [95]:
cmp_plot(history_list1, model_list1, 'precision', 'Trianing_precision_acc_cmp', epoch_num, validation=False)

In [96]:
cmp_plot(history_list1, model_list1, 'precision', 'validation_precision_acc_cmp', epoch_num, train=False)

In [97]:
cmp_plot(history_list1, model_list1, 'recall', 'Trianing_validation_acc_cmp', epoch_num, validation=False)

In [98]:
cmp_plot(history_list1, model_list1, 'recall', 'validation_acc_cmp', epoch_num, train=False)

In [99]:
for image, lab in test_data_generator:
    cmp2 = pd.DataFrame({'D' : model1.evaluate(image, lab),
                         'DX' : model2.evaluate(image, lab),
                         'DXR' : model3.evaluate(image, lab),
                        'DXR_BiGRU' : model4.evaluate(image, lab),
                        'DXR_BiLSTM' : model5.evaluate(image, lab),
                        'D_BiGRU' : model6.evaluate(image, lab),
                        'D_BiLSTM' : model7.evaluate(image, lab),
                        'DX_BiGRU' : model8.evaluate(image, lab),
                        'DX_BiLSTM' : model9.evaluate(image, lab)},
                        index=['Loss', 'Accuracy', 'Precision', 'Recall'])
    break

cmp2